In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('CarPrice.csv')
df.drop(columns=['car_ID','CarName'],inplace=True)
df

In [ ]:
con_col,cate_col = [],[]              #แยก column ที่เป็น เลข กับ category
for i in df.columns:
    if np.issubdtype(df[i],np.number):
        con_col.append(i)
    else:
        cate_col.append(i)
len(con_col)+len(cate_col)
# price เป็น Y
con_col.remove('price')
con_col

In [ ]:
std_scale = preprocessing.StandardScaler()
cor_df = std_scale.fit_transform(df[con_col])
cor_df = pd.DataFrame(cor_df,columns=con_col)
lower = pd.DataFrame(np.tril(cor_df.corr(),-1),columns = cor_df.columns)
to_drop = [column for column in lower.columns if any(lower[column] > 0.86)]
to_drop


In [ ]:
cor_df.drop(columns=to_drop,inplace=True)
cor_df

In [ ]:
df_encode = pd.get_dummies(data=df[cate_col],columns= cate_col,drop_first=True)
df_encode

In [ ]:
#standard เฉพาะ continuous
X_standard = std_scale.fit_transform(cor_df)
X_standard = pd.DataFrame(X_standard,columns=cor_df.columns)
#รวม dataframe continuous(หลังstandardize) กับ category
df_afterEncode = pd.concat([X_standard,df_encode], axis=1, join='inner') 
column_afterEncode_temp = df_afterEncode.columns
df_afterEncode


In [ ]:
pca = PCA()
X_pca = pca.fit_transform(df_afterEncode)
fig = plt.figure(figsize =(10, 7))
plt.bar([i for i in range(1,len(pca.explained_variance_ratio_)+1)],pca.explained_variance_ratio_,0.75)  #eigen value มี 39 ตัว เพราะมี 39 columns
plt.ylabel("Explained variance ratio",fontsize = 12) 


In [ ]:
pca2 = PCA(n_components=5)
X_pca_2 = pca2.fit_transform(X_standard)

In [ ]:
#shuffle spilt
Rseed = 55
x_train_set , x_test, y_train_set , y_test = train_test_split(X_standard,df['price'],test_size= 0.3,random_state= Rseed)
x_train,x_validate,y_train,y_validate =     train_test_split(x_train_set,y_train_set,test_size=0.3 ,random_state= Rseed)


In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_validate)
y_test_pred_lr = lr.predict(x_test)

In [ ]:
no_pca_r2_valid = r2_score(y_pred_lr,y_validate)
no_pca_r2_test = r2_score(y_test_pred_lr,y_test)
no_pca_mse_valid = mean_squared_error(y_pred_lr,y_validate)
no_pca_mse_test = mean_squared_error(y_test_pred_lr,y_test)
print(no_pca_r2_valid)    
print(no_pca_r2_test)  
print(no_pca_mse_valid)   
print(no_pca_mse_test)

In [ ]:
r2_valid_score = [no_pca_r2_valid]
r2_test_score = [no_pca_r2_test]
mse_valid_score = [no_pca_mse_valid]
mse_test_score = [no_pca_mse_test]
score_labelx = ['no PCA']
x_pca_list = []
n_compo = []
for i in range (4,25,5):
    pca = PCA(n_components=i)
    X_pca = pca.fit_transform(df_afterEncode)
    x_pca_list.append(X_pca)
    n_compo.append(i)
   
for i in range (0,len(x_pca_list)):  
    x_train_set , x_test, y_train_set , y_test  =     train_test_split(x_pca_list[i],df['price'],test_size= 0.3,random_state= Rseed) 
    x_train,x_validate,y_train,y_validate       =     train_test_split(x_train_set,y_train_set,test_size=0.3,random_state= Rseed) 
    lr = LinearRegression()    
    lr.fit(x_train, y_train)
    y_pred_lr       =   lr.predict(x_validate)      # valid y_pred_lr       y_validate
    y_test_pred_lr  =   lr.predict(x_test)     # test  y_test_pred_lr  y_test
        # print(f'r2_pred  = {r2_score(y_pred_lr,y_validate)}')    
        # print(f'r2_test  = {r2_score(y_test_pred_lr,y_test)}')  # y_test_pred_lr ค่าที่ได้จาก model  , y_test ค่าที่แยกไว้ test
        # print(f'mse_pred = {mean_squared_error(y_pred_lr,y_validate)}')   
        # print(f'mse_test = {mean_squared_error(y_test_pred_lr,y_test)}')
        # print(f'n_component = {n_compo[i]}')
        # print('--------------------------------------')

    r2_valid_score.append(r2_score(y_pred_lr,y_validate))
    r2_test_score.append(r2_score(y_test_pred_lr,y_test))
    mse_valid_score.append(mean_squared_error(y_pred_lr,y_validate))
    mse_test_score.append (mean_squared_error(y_test_pred_lr,y_test))
    score_labelx.append(f'PCA_{n_compo[i]}')  
          
    # fig = plt.figure(figsize =(10, 7))
    # plt.bar(score_labelx,r2_valid_score,0.75)
    # plt.plot(r2_test_score,color='r')

    # fig = plt.figure(figsize =(10, 7))
    # plt.bar(score_labelx,mse_valid_score,0.75)
    # plt.plot(mse_test_score,color='r')

    # score_labelx.clear()
    # r2_test_score.clear()
    # r2_valid_score.clear()
    # mse_test_score.clear()
    # mse_valid_score.clear()



In [ ]:
fig = plt.figure(figsize =(10, 7))
plt.bar(score_labelx,r2_valid_score ,0.75)
plt.plot(r2_test_score,color='r')

In [ ]:
fig = plt.figure(figsize =(10, 7))
plt.bar(score_labelx, mse_valid_score,0.75)
plt.plot(mse_test_score,color='r')